In [18]:
# imports

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

# Constants
MODEL = "llama3.2"
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}


ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')



class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)
        
# A function that writes a User Prompt that asks for summaries of websites:
def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

def summarize(url):
    website = Website(url)
    response = ollama_via_openai.chat.completions.create(
        model = MODEL,
        messages = messages_for(website)
    )
    return response.choices[0].message.content


# A function to display this nicely in the Jupyter output, using markdown
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))
    
display_summary("https://edwarddonner.com")
display_summary("https://google.com")


# Website Summary

The website is owned by Edward Donner, a co-founder and CTO of Nebula.io. The platform leverages Large Language Models (LLMs) to help people discover their potential.

## Recent News and Announcements

* **January 23, 2025:** LLM Workshop – Hands-on with Agents – resources
* **December 21, 2024:** Welcome, SuperDataScientists!
* **November 13, 2024:** Mastering AI and LLM Engineering – Resources
* **October 16, 2024:** From Software Engineer to AI Data Scientist – resources

This website primarily focuses on Edward Donner's personal story, industry involvement with AI, and the work of his company Nebula.io.

# Google Website Summary

## Overview
The Google website appears to be a general overview page with various links that take users to different sections of the site.

### Notable Sections:

* **News/Announcements**: Google's commitment to climate action is showcased through a statement promoting their third decade of sustainability efforts.
### Content:
There are no notable updates or announcements on current news, and only minor configuration options like default dark theme settings.

## Key Features/Topics

* Search engine services
* Productivity tools (e.g. Gmail)
* Advertising and business solutions
* Artificial intelligence capabilities (mentioned in "How Search works" and other relevant sections)

Note that this summary is based on the provided content and might not be comprehensive, as some sections like advertising may require further examination for specific product offerings or updates.